In [3]:
from kaggle_environments import get_episode_replay, list_episodes, list_episodes_for_team, list_episodes_for_submission
import os
import requests
import json
import pandas as pd
import time
import shutil
import numpy as np

In [3]:
# 1. Get the ids of teams and corresponding replays
# 2. Save replay from top 5 teams at path.
#TODO: Consider filtering the saved replays based on reward

In [4]:
path = "data/"
eval_path = "data/eval/"
test_path = "data/test/"
train_path = "data/train/"
base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

In [4]:
r = requests.post(list_url, json = {"teamId":  4820508})
r = r.json()

In [11]:
teams_df = pd.DataFrame(r['result']['teams'])
teams_df.sort_values('publicLeaderboardRank', inplace = True)
teams_df.head()

,id,teamName,competitionId,teamLeaderId,isBenchmark,createdAfterDeadline,medal,dateMedalAwarded,submissionCount,lastSubmissionDate,publicLeaderboardSubmissionId,publicLeaderboardScore,publicLeaderboardRank,privateLeaderboardSubmissionId,privateLeaderboardScore,privateLeaderboardRank,competition,teamMembers,inboundMergeRequests,outboundMergeRequests
628,5234835,convexOptimization,18011,672770,False,False,0,None,166,"{'seconds': 1599824450, 'nanos': 530000000}",17050271.0,None,1.0,None,None,None,None,"[{'id': 672770, 'displayName': 'HyperSeedOptim...",[],[]
9,4714287,Tom Van de Wiele,18011,582611,False,False,0,None,7,"{'seconds': 1599505053, 'nanos': 60000000}",17114281.0,None,2.0,None,None,None,None,"[{'id': 582611, 'displayName': 'Tom Van de Wie...",[],[]
157,5118779,mzotkiew,18011,80203,False,False,0,None,29,"{'seconds': 1599774490, 'nanos': 903333300}",17041870.0,None,3.0,None,None,None,None,"[{'id': 80203, 'displayName': 'mzotkiew', 'ema...",[],[]
290,5133228,Robiland,18011,5330482,False,False,0,None,79,"{'seconds': 1599793515, 'nanos': 576666700}",17092555.0,None,4.0,None,None,None,None,"[{'id': 5330482, 'displayName': 'PS', 'email':...",[],[]
161,5119134,Fei Wang,18011,171088,False,False,0,None,184,"{'seconds': 1599820416, 'nanos': 513333300}",17047360.0,None,5.0,None,None,None,None,"[{'id': 171088, 'displayName': 'Fei Wang', 'em...",[],[]


In [3]:
episodes = list_episodes_for_team(5234835)

In [43]:
print(type(episodes))
print(episodes.keys())

<class 'dict'>
dict_keys(['result', 'wasSuccessful'])


In [4]:
player1_episode_ids = [episodes["result"]["episodes"][i]["id"] for i in range(len(episodes["result"]["episodes"]))]
len(player1_episode_ids)

77974

In [5]:
# get the agent ids for team: 5234835
def get_agent_id_for_team(team_id, episodes):
    agent_ids = {}
    for episode in episodes["result"]["episodes"]:
        for agent in episode['agents']:
            if agent['submission']['teamId'] == team_id:
                #print(episode["id"])
                agent_ids[str(episode["id"])] = agent["index"]
    
    with open(path+'agent_ids_'+str(team_id)+'.txt', 'w') as outfile:
        json.dump(agent_ids, outfile)
    return agent_ids

In [6]:
agent_ids = get_agent_id_for_team(5234835, episodes)

In [8]:
episode_id = player1_episode_ids[0]
data = get_episode_replay(episode_id)
with open(path+'convexOptimization_'+str(episode_id)+'.txt', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
replay = get_episode_replay(1375512)

In [11]:
for i in range(15260,15260+1000,60):
    for j in range(i,i+60):
        episode_id = player1_episode_ids[j]
        replay = get_episode_replay(episode_id)
        with open(path+'convexOptimization_'+str(episode_id)+'.txt', 'w') as outfile:
            json.dump(replay, outfile)
    time.sleep(60)

In [14]:
files = os.listdir(path)
len(files)

15596

In [5]:
# delete unsuccessful replays
num_deleted = 0
for file in files:
    replay_path = os.path.join(path, file)
    if os.path.isdir(replay_path) == True: continue
    with open(replay_path) as replay_file:
        replay = json.load(replay_file)
    if replay['wasSuccessful'] == False:
        num_deleted+=1
        os.remove(replay_path)
print(num_deleted)

0


In [13]:
eval_size = 3256
test_size = 3256
train_size = 9766

In [8]:
files = os.listdir(path)
while eval_size > 0:
    file = files[np.random.randint(0,len(files))]
    try:
        if os.path.isdir(os.path.join(path, file)) == True: continue
        shutil.move(os.path.join(path, file), os.path.join(eval_path, file))
        eval_size -= 1
    except: continue
assert (len(os.listdir(eval_path)) == 3256)

In [11]:
files = os.listdir(path)
while test_size > 0:
    file = files[np.random.randint(0,len(files))]
    try:
        if os.path.isdir(os.path.join(path, file)) == True: continue
        shutil.move(os.path.join(path, file), os.path.join(test_path, file))
        test_size -= 1
    except: continue
assert (len(os.listdir(test_path)) == 3256)

In [14]:
files = os.listdir(path)
while train_size > 0:
    file = files[np.random.randint(0,len(files))]
    try:
        if os.path.isdir(os.path.join(path, file)) == True: continue
        shutil.move(os.path.join(path, file), os.path.join(train_path, file))
        train_size -= 1
    except: continue
assert (len(os.listdir(train_path)) == 9766)